# Overview of all Spectro data including few images

- Goal verification 
- Restart the Kernel first

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/in2p3/CNRS
- creation date : 2025-01-22 (w_2024_50)
- update : 2025-01-22 : Select images at low altitude or high airmass

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm, SymLogNorm
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.colors as mcolors
import matplotlib.cm as cmx
import matplotlib.cm as cm


%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec
from spectractor.tools import from_lambda_to_colormap, wavelength_to_rgb

# %matplotlib widget
import h5py
from scipy import interpolate
from astropy.time import Time
from datetime import datetime, timedelta
import seaborn as sns

from itertools import cycle, islice
import os

In [ ]:
plt.rcParams["figure.figsize"] = (18, 8)
plt.rcParams["axes.labelsize"] = "xx-large"
plt.rcParams["axes.titlesize"] = "xx-large"
plt.rcParams["xtick.labelsize"] = "xx-large"
plt.rcParams["ytick.labelsize"] = "xx-large"
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["font.size"] = 12

In [ ]:
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator

from astropy.visualization import (
    MinMaxInterval,
    SqrtStretch,
    ZScaleInterval,
    PercentileInterval,
    ImageNormalize,
    imshow_norm,
)
from astropy.visualization.stretch import (
    SinhStretch,
    LinearStretch,
    AsinhStretch,
    LogStretch,
)

transform = AsinhStretch() + PercentileInterval(99.0)

from astropy import units as u
from astropy.coordinates import Angle

In [ ]:
from lsst.daf.butler import Butler
import astropy.units as u
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 99)
pd.set_option("display.max_row", 25)
from astropy.time import Time

import scipy.stats

import matplotlib

%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
# LSST Display
# The advantage to use firefly is that the firefly display can handle a lotsof images
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend("firefly")

In [ ]:
# import my tool library for spectra
from auxtelana2025.spectrom import libanaspectra as myspeclib
# dir(myspeclib)

## Configuration

- Use the tag report to guess the collections in which we exect data :  https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport

In the case of LSSTComCam:

- https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport?day_obs=2024-10-24&instrument=LSSTComCam&repo=embargo_new&collection=LSSTComCam%2Fprompt%2Foutput-2024-10-24&collection_sky=LSSTComCamSim%2Fruns%2FDRP%2FOR4%2Fw_2024_25%2FDM-45066&skymap_name=ops_rehersal_prep_2k_v1&col_sciprog=science_program&col_target=target&col_filter=filter&col_id=id&ts_hide_code=1

In [ ]:
instrument = "LATISS"
# collection = 'u/dagoret/auxtel_atmosphere_202501_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_w_2024_50_oldptc-test20250109b'
collection = "u/dagoret/auxtel_atmosphere_202501_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_w_2024_50_newptc-test20250110a"
# 2025-01-21
collection = "u/dagoret/auxtel_atmosphere_202501_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_No5SigmaClip_w_2024_50_test20250121a"
# 20250121T124042Z

In [ ]:
# import lsst.daf.butler as dafButler
import lsst.summit.utils.butlerUtils as butlerUtils

butler = butlerUtils.makeDefaultLatissButler(embargo=True)

repo = "/repo/main"
repo = "/repo/embargo"
# repo = "LATISS"
# butler = dafButler.Butler(repo)
registry = butler.registry

## Query about the collections available

- select collections LATISS

In [ ]:
for c in sorted(registry.queryCollections()):
    # if "u/jneveu" in c and "auxtel_atmo" in c and "SensorFlat" in c and "FixA1" in c:
    # if "u/dagoret" in c and "auxtel_atmo" in c and "SensorFlat" in c and "FixA1" in c:
    if "u/dagoret" in c and "auxtel_atmo":
        print(c)

In [ ]:
for _ in butler.registry.queryCollections():
    if "LATISS/" in _:
        print(_)

- From the list available today , we probably have to look inside the colleciton `LSSTComCam/nightlyValidation`

## Create the butler on the selected collection

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collection).any(
        execute=False, exact=False
    ):
        # Limit search results to the data products
        if (
            ("_config" not in datasetType.name)
            and ("_log" not in datasetType.name)
            and ("_metadata" not in datasetType.name)
            and ("_resource_usage" not in datasetType.name)
        ):
            print(datasetType)

- Notice no visit-Table is available. Thus to know which visiId are existing, we will use the registry later

## Extract the list of visitId from the butler's registry 

### Get the list of information that can be extracted from the registryfor each exposure

In [ ]:
print(registry.dimensions["exposure"].RecordClass.fields)

### Create the pandas dataframe from the information inside the butler's registry

In [ ]:
df_exposure = pd.DataFrame(
    columns=[
        "id",
        "obs_id",
        "day_obs",
        "seq_num",
        "time_start",
        "time_end",
        "type",
        "target",
        "filter",
        "zenith_angle",
        "expos",
        "ra",
        "dec",
        "skyangle",
        "azimuth",
        "zenith",
        "science_program",
        "jd",
        "mjd",
    ]
)

### Fill the pandas dataframe from the records inside the butler's registry for each exposure having a datasets='spectractorSpectrum'

In that wayn the filling of the pandas dataframe logbook is faster.

In [ ]:
where_expr = "instrument='LATISS'"
records = list(
    butler.registry.queryDimensionRecords(
        "visit",
        datasets="spectractorSpectrum",
        where=where_expr,
        collections=collection,
    )
)
refs = list(
    set(
        butler.registry.queryDatasets(
            "spectractorSpectrum", where=where_expr, collections=collection
        )
    )
)

In [ ]:
print(
    f">>> Number of reconstructed spectra : {len(records)} \n in collection {collection}"
)
records[0]

In [ ]:
def get_whereclause(expos_id):
    return f"exposure.id = {expos_id} and instrument = 'LATISS'"

In [ ]:
# for count, info in enumerate(registry.queryDimensionRecords('exposure',where = where_expr)):
list_of_ids = np.zeros(len(records)).astype(int)
# first loop on records 'spectractorSpectrum'
for count, r in enumerate(records):
    the_whereclause = get_whereclause(r.id)
    list_of_ids[count] = r.id
    # then select in the registry the entry with the proper id
    for count2, info in enumerate(
        registry.queryDimensionRecords("exposure", where=the_whereclause)
    ):
        try:
            # decode entry in the registry
            id_ = info.id
            obs_id_ = info.obs_id
            day_obs_ = info.day_obs
            seq_num_ = info.seq_num
            timespan_ = info.timespan

            timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
            timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
            timespan_jd_ = timespan_.begin.jd
            timespan_mjd_ = timespan_.begin.mjd

            df_exposure.loc[count] = [
                info.id,
                info.obs_id,
                info.day_obs,
                info.seq_num,
                timespan_begin_,
                timespan_end_,
                info.observation_type,
                info.target_name,
                info.physical_filter,
                info.zenith_angle,
                info.exposure_time,
                info.tracking_ra,
                info.tracking_dec,
                info.sky_angle,
                info.azimuth,
                info.zenith_angle,
                info.science_program,
                timespan_jd_,
                timespan_mjd_,
            ]

        except:
            # print(">>>   Unexpected error:", sys.exc_info()[0])
            # info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
            # info_timespan_end_to_string = "2051-01-01 00:00:00.00"

            info_timespan_begin_to_string = info.group
            info_timespan_end_to_string = info.group

            info_timespan_begin_mjd = Time(info_timespan_begin_to_string).mjd
            info_timespan_begin_jd = Time(info_timespan_begin_to_string).jd

            # print("\t >>>> after error try to recover mjd and jd == > mjd, jd = ",info_timespan_begin_mjd ,info_timespan_begin_jd )

            # info_timespan_begin_jd = 0
            # info_timespan_begin_mjd = 0
            df_exposure.loc[count] = [
                info.id,
                info.obs_id,
                info.day_obs,
                info.seq_num,
                pd.to_datetime(info_timespan_begin_to_string),
                pd.to_datetime(info_timespan_end_to_string),
                info.observation_type,
                info.target_name,
                info.physical_filter,
                info.exposure_time,
                info.tracking_ra,
                info.tracking_dec,
                info.sky_angle,
                info.azimuth,
                info.zenith_angle,
                info.science_program,
                info_timespan_begin_jd,
                info_timespan_begin_mjd,
            ]

        if count < 3:
            print(
                "-----------------------------------------------------",
                count,
                "---------------------------------------------------------",
            )
            print(info)
            print("\t id:                  ", info.id)
            print("\t day_obs:             ", info.day_obs)
            print("\t info_group.          ", info.group)
            print("\t seq_num:             ", info.seq_num)
            print("\t type-of-observation: ", info.observation_type)
            print("\t target:              ", info.target_name)

            timespan_ = info.timespan
            timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
            timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
            timespan_jd_ = timespan_.begin.jd
            timespan_mjd_ = timespan_.begin.mjd

            mjd = timespan_mjd_
            jd = timespan_jd_
            print("MJD,JD : ", mjd, jd)

#### convert some columns into integer

In [ ]:
df_exposure = df_exposure.astype(
    {"id": int, "day_obs": int, "seq_num": int, "jd": int, "mjd": int}
)

### Find the different type of exposures

- By definition 'spectractorSpectrum' are science type

In [ ]:
df_exposure["type"].unique()

### Select the science exposure

In [ ]:
# select just in case there is no pre-selection of 'spectractorSpectrum'
selection_criteria = (df_exposure.type == "science") & (
    df_exposure.science_program == "spec-survey"
)
df_science = df_exposure[selection_criteria]

In [ ]:
# df_science

In [ ]:
df_science.columns

## Add missing info on airmass

In [ ]:
# Angle(45*u.deg).to("rad").value

In [ ]:
# df_science.assign(lambda df: 1./np.cos(Angle(df['zenith_angle']*u.deg).to("rad").value));
pd.options.mode.chained_assignment = None
df_science.loc[:, ["airmass"]] = df_science["zenith_angle"].apply(
    lambda x: 1.0 / np.cos(Angle(x * u.deg).to("rad").value)
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 6), layout="constrained")
df_science.plot(x="mjd", y="airmass", ax=ax1, marker=".", c="r", lw=0, grid=True)
df_science.plot(x="day_obs", y="airmass", ax=ax2, marker=".", c="b", lw=0, grid=True)

ax1.yaxis.set_inverted(True)
ax2.yaxis.set_inverted(True)
ax1.set_ylabel("airmass")
ax2.set_ylabel("airmass")

## Selection

In [ ]:
# flag_selection = (df_science.day_obs == 20250108) & (df_science["filter"] == 'OG550_65mm_1~holo4_003')
# flag_selection = (df_science.day_obs == 20250108) & (df_science["filter"] == 'BG40_65mm_1~holo4_003')
# flag_selection = (df_science.day_obs == 20250109)
flag_selection = (
    df_science.day_obs == 20250116
)  # Select the night with the highest airmass

In [ ]:
pd.options.display.max_rows = 999
df_science_day = df_science[flag_selection]
df_science_day = df_science_day.reset_index(drop=True)
df_science_day

In [ ]:
df_sel = df_science_day[df_science_day["airmass"] > 2.1]

list_of_visits = list(df_sel.id.values)

In [ ]:
list_of_visits

In [ ]:
def GetListOfSpectra(butler, list_of_visits, the_collection):
    """
    Retrieve a list of Spectra:
    Parameters:
      butler : the butler
      list_of_visits : the list of visit id
      collection : the collection

    Returns:
     all_params_spectrum = []
     all_params_spectrogram = []
     all_visitid = []
     all_headers = []
     all_spectra = []
    """

    # container contining the data
    all_params_spectrum = []
    all_params_spectrogram = []
    all_visitid = []
    all_headers = []
    all_spectra = []

    # loop on visits
    for idx, visitid in enumerate(list_of_visits):
        try:
            spec = butler.get(
                "spectractorSpectrum",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_headers.append(spec.header)
            all_spectra.append(spec)
            p = butler.get(
                "spectrumLibradtranFitParameters",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_params_spectrum.append(p)
            p = butler.get(
                "spectrogramLibradtranFitParameters",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_params_spectrogram.append(p)
            all_visitid.append(visitid)
        # except ValueError:
        except Exception as inst:
            except_type = type(inst)
            except_args = inst.args
            print(
                "catch exception ", inst, "type =", except_type, "args = ", except_args
            )
            print("\t >>>>> Skip visitid ", visitid)
            continue
    return (
        all_visitid,
        all_headers,
        all_params_spectrum,
        all_params_spectrogram,
        all_spectra,
    )

In [ ]:
(
    all_visitid,
    all_headers,
    all_params_spectrum,
    all_params_spectrogram,
    all_spectra,
) = GetListOfSpectra(butler, list_of_visits, collection)

## Example for viewing one image

### Select a date and a visitId

In [ ]:
index_sel = 0

DATEOBS = df_science_day.iloc[index_sel]["day_obs"]
visitId = df_science_day.iloc[index_sel]["id"]

In [ ]:
print(f"Selected observation {visitId} for date {DATEOBS}")

### Extract the exposures postISRCCD from the selected visitID

In [ ]:
where_clause = f"instrument='LATISS' AND exposure.day_obs={DATEOBS}"
dataId = {"visit": visitId, "instrument": instrument}
datasetRefs = registry.queryDatasets(
    "postISRCCD", dataId=dataId, collections=collection
)
# one dictionnary for the focal surface
title_dict = {}
postisr_dict = {}

for i, ref in enumerate(datasetRefs):
    exposure = ref.dataId["exposure"]  # for postISRCCD
    # exposure = ref.dataId["visit"]  For calexp
    detector = ref.dataId["detector"]
    physical_filter = ref.dataId["physical_filter"]
    postisrccd = butler.get(ref)
    print(ref.dataId, postisrccd)
    the_title = f"id : {exposure}, det = {detector}, b = {physical_filter}"
    title_dict[detector] = the_title
    postisr_dict[detector] = postisrccd
N = len(title_dict)
print(f"Number of images = {N}")

## View all the images of the focal surface for that visit

- would need the mosaic for the focal surface

In [ ]:
for count in range(N):
    display = afwDisplay.Display(frame=count)
    display.scale("asinh", "zscale")
    display.mtv(postisr_dict[count].image, title=title_dict[count])

## Find the spectrum

In [ ]:
import spectractor
from spectractor.tools import from_lambda_to_colormap, wavelength_to_rgb
from spectractor.simulation.throughput import (
    load_transmission,
    plot_transmission_simple,
    TelescopeTransmission,
)
from spectractor import parameters
from spectractor.extractor import dispersers
from spectractor.config import load_config, set_logger
import os, sys, re

### Find the spectrum

In [ ]:
the_specref_index = np.where(list_of_ids == visitId)[0][0]

In [ ]:
the_specref_index
the_specref = refs[the_specref_index]

In [ ]:
def GetListOfSpectra(butler, list_of_visits, the_collection):
    """
    Retrieve a list of Spectra:
    Parameters:
      butler : the butler
      list_of_visits : the list of visit id
      collection : the collection

    Returns:
     all_params_spectrum = []
     all_params_spectrogram = []
     all_visitid = []
     all_headers = []
     all_spectra = []
    """

    # container contining the data
    all_params_spectrum = []
    all_params_spectrogram = []
    all_visitid = []
    all_headers = []
    all_spectra = []

    # loop on visits
    for idx, visitid in enumerate(list_of_visits):
        try:
            spec = butler.get(
                "spectractorSpectrum",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_headers.append(spec.header)
            all_spectra.append(spec)
            p = butler.get(
                "spectrumLibradtranFitParameters",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_params_spectrum.append(p)
            p = butler.get(
                "spectrogramLibradtranFitParameters",
                visit=visitid,
                collections=the_collection,
                detector=0,
                instrument="LATISS",
            )
            all_params_spectrogram.append(p)
            all_visitid.append(visitid)
        # except ValueError:
        except Exception as inst:
            except_type = type(inst)
            except_args = inst.args
            print(
                "catch exception ", inst, "type =", except_type, "args = ", except_args
            )
            print("\t >>>>> Skip visitid ", visitid)
            continue
    return (
        all_visitid,
        all_headers,
        all_params_spectrum,
        all_params_spectrogram,
        all_spectra,
    )

In [ ]:
list_visitid = [visitId]
(
    all_foundvisitid,
    all_headers,
    all_params_spectrum,
    all_params_spectrogram,
    all_spectra,
) = GetListOfSpectra(butler, list_visitid, collection)

### Plot the spectrum

- see https://github.com/sylvielsstfr/LSST-Rehearsal2024/blob/13-analyse-my-bps-run-with-spectro-sept24/notebooks/SpectroTuto/StudySpectraOneNight-savehdf5.ipynb

In [ ]:
def plot_spectra(spectra, colorparams, collection, dateobs, figsize=(10, 6)):
    """
    plot spectra
    """

    # colormap = cm.Reds
    colormap = cm.jet

    normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))

    all_target_names = []

    fig = plt.figure(figsize=figsize)
    count = 0
    for spec in spectra:
        target_name = spec.target.label
        if target_name in all_target_names:
            plt.plot(spec.lambdas, spec.data, color=colormap(normalize(spec.airmass)))
        else:
            plt.plot(
                spec.lambdas,
                spec.data,
                color=colormap(normalize(spec.airmass)),
                label=target_name,
            )
            all_target_names.append(target_name)
        count += 1

    plt.grid()
    plt.xlabel("$\lambda$ [nm]")
    plt.ylabel(f"Flux [{spec.units}]")
    plt.legend()

    ax = plt.gca()

    # Colorbar setup
    s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
    s_map.set_array(colorparams)

    # If color parameters is a linspace, we can set boundaries in this way
    halfdist = (colorparams[1] - colorparams[0]) / 2.0
    boundaries = np.linspace(
        colorparams[0] - halfdist, colorparams[-1] + halfdist, len(colorparams) + 1
    )

    # Use this to emphasize the discrete color values
    cbar = fig.colorbar(
        s_map, ax=ax
    )  # , spacing='proportional', ticks=colorparams, boundaries=boundaries, format='%2.2g') # format='%2i' for integer

    # Use this to show a continuous colorbar
    # cbar = fig.colorbar(s_map, spacing='proportional', ticks=colorparams, format='%2i')
    cbar.set_label("Airmass $z$")
    title = f"Observations : {dateobs}, nspec = {count}"
    suptitle = f"collection = {collection}"
    plt.title(title)
    plt.suptitle(suptitle, fontsize=10)
    # plt.tight_layout()
    plt.show()
    return fig


def plot_spectra_ax(spectra, ax, colorparams, dateobs):
    """
    plot spectra
    """

    # colormap = cmx.Reds
    colormap = cmx.jet

    normalize = mcolors.Normalize(vmin=np.min(colorparams), vmax=np.max(colorparams))

    all_target_names = []

    count = 0
    for spec in spectra:
        target_name = spec.target.label
        if target_name in all_target_names:
            ax.plot(spec.lambdas, spec.data, color=colormap(normalize(spec.airmass)))
        else:
            ax.plot(
                spec.lambdas,
                spec.data,
                color=colormap(normalize(spec.airmass)),
                label=target_name,
            )
            all_target_names.append(target_name)
        count += 1

    ax.grid()
    ax.set_xlabel("$\lambda$ [nm]")
    ax.set_ylabel(f"Flux [{spec.units}]")
    ax.legend()

    # ax = plt.gca()

    # Colorbar setup
    s_map = cm.ScalarMappable(norm=normalize, cmap=colormap)
    s_map.set_array(colorparams)

    # If color parameters is a linspace, we can set boundaries in this way

    halfdist = (colorparams[1] - colorparams[0]) / 2.0
    boundaries = np.linspace(
        colorparams[0] - halfdist, colorparams[-1] + halfdist, len(colorparams) + 1
    )

    # Use this to emphasize the discrete color values
    fig = ax.figure
    cbar = fig.colorbar(
        s_map, ax=ax
    )  # , spacing='proportional', ticks=colorparams, boundaries=boundaries, format='%2.2g') # format='%2i' for integer
    # cbar = ax.collections[-1].colorbar

    # Use this to show a continuous colorbar
    # cbar = fig.colorbar(s_map, spacing='proportional', ticks=colorparams, format='%2i')
    cbar.set_label("Airmass $z$")
    title = f"Observations : {dateobs}, nspec = {count}"
    # ax.set_title(title)
    # plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10), layout="constrained")
# plot_spectra_ax(all_spectra, ax, [spec.airmass for spec in all_spectra],dateobs=DATEOBS)
all_spectra[0].plot_spectrum(ax=ax1)
all_spectra[0].plot_spectrogram(ax=ax2)
ax1.set_title(f"Observed spectra {visitId} for night {DATEOBS}")

In [ ]:
all_spectra[0].plot_spectrum_summary()

In [ ]:
assert False

In [ ]:
display.clearViewer()
afwDisplay.setDefaultBackend("firefly")

In [ ]:
list_of_visits

# Example for all visits

In [ ]:
FLAG_ONE_VISIT = False

In [ ]:
if FLAG_ONE_VISIT:
    index = 0
    visitId = list_of_visits[index]
    dataId = {"visit": visitId, "instrument": instrument}
    datasetRefs = registry.queryDatasets(
        "postISRCCD", dataId=dataId, collections=collection
    )
    exposure = ref.dataId["exposure"]
    detector = ref.dataId["detector"]
    physical_filter = ref.dataId["physical_filter"]
    postisrccd = butler.get(ref)
    the_title = f"id : {exposure}, det = {detector}, b = {physical_filter}"
    display = afwDisplay.Display(frame=count)
    display.scale("asinh", "zscale")
    display.mtv(postisrccd.image, title=the_title)

In [ ]:
# list_of_visitId = list(df_science.id)
if not FLAG_ONE_VISIT:
    list_of_visitId = list_of_visits
    image_count = 0
    # loop on visit id
    for idcount, visitId in enumerate(list_of_visitId):
        dataId = {"visit": visitId, "instrument": instrument}
        print(dataId)
        datasetRefs = registry.queryDatasets(
            "postISRCCD", dataId=dataId, collections=collection
        )

        # loop on image on the focal surface
        for i, ref in enumerate(datasetRefs):
            exposure = ref.dataId["exposure"]
            detector = ref.dataId["detector"]
            physical_filter = ref.dataId["physical_filter"]
            postisrccd = butler.get(ref)
            the_title = f"id : {exposure}, det = {detector}, b = {physical_filter}"
            display = afwDisplay.Display(frame=idcount)
            display.scale("asinh", "zscale")
            display.mtv(postisrccd.image, title=the_title)
            image_count += 1

In [ ]:
N